In [1]:
import librosa
import librosa.display
import matplotlib.pyplot as plt
%matplotlib nbagg
import numpy as np

In [2]:
import os

In [3]:
os.environ['KERAS_BACKEND'] = 'tensorflow'

In [4]:
import keras as K

Using TensorFlow backend.


In [5]:
import jams
import jams.display

In [6]:
import imp
import pumpp
import pickle

In [7]:
mod = imp.load_source('train_model', '/home/bmcfee/git/ismir2017_chords/code/train_model.py')

In [8]:
dmod = imp.load_source('train_deep', '/home/bmcfee/git/ismir2017_chords/code/train_deep.py')

In [9]:
with open('/home/bmcfee/git/ismir2017_chords/data/pump.pkl', 'rb') as fd:
    pump = pickle.load(fd)

/home/bmcfee/miniconda/envs/py35/lib/python3.5/site-packages/sklearn/base.py:312: UserWarning: Trying to unpickle estimator LabelEncoder from version 0.18 when using version 0.19.0. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/home/bmcfee/miniconda/envs/py35/lib/python3.5/site-packages/sklearn/base.py:312: UserWarning: Trying to unpickle estimator MultiLabelBinarizer from version 0.18 when using version 0.19.0. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [10]:
cr1s = mod.construct_model(pump, True)[0]

In [11]:
cr2s = dmod.construct_model(pump, True)[0]

In [19]:
cr1s.load_weights('/home/bmcfee/git/ismir2017_chords/data/model_struct_aug/fold00_weights.pkl')

In [20]:
cr2s.load_weights('/home/bmcfee/git/ismir2017_chords/data/model_deep_struct_aug/fold00_weights.pkl')

In [53]:
import scipy.stats

In [119]:
import pandas as pd

In [55]:
import mir_eval

In [194]:
from librosa import time_to_frames

SEMITONE_TO_SCALE_DEGREE = ['1', 'b2', '2', 'b3', '3', '4', 'b5', '5', 'b6', '6', 'b7', '7']

def preds_to_inversions(pump_op, tag_pred, bass_pred):
    
    #ann = pump['chord_tag'].inverse(tag_pred)
    ann = pump_op.inverse(tag_pred)
    
    for obs in ann.pop_data():
        value = obs.value
        
        if obs.value not in ('N', 'X'):
            # 1. Convert (time, time+duration) to frames
            start, end = time_to_frames([obs.time, obs.time + obs.duration],
                                        sr=pump_op.sr, hop_length=pump_op.hop_length)
            
            # 2. G-mean aggregate bass_pred at the frame level
            mean_bass = scipy.stats.gmean(bass_pred[start:end+1])
        
            # 3. Find the argmax of the aggregated frames
            bass_pc = np.argmax(mean_bass)
            
            # 4. map argmax to scale degree
            root_pc, pitches, _ = mir_eval.chord.encode(obs.value)
            
            bass_rel = 0
            if bass_pc < 12:
                bass_rel = np.mod(bass_pc - root_pc, 12)
            
            if bass_rel and pitches[bass_rel]:
                value = '{}/{}'.format(value, SEMITONE_TO_SCALE_DEGREE[bass_rel])
            
        # 5. Re-insert to ann
        ann.append(time=obs.time, duration=obs.duration,
                   value=value,
                   confidence=obs.confidence)
        
    return ann

In [149]:
mir_eval.chord.encode('C#:maj/3')

(1, array([1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0]), 4)

In [196]:
model = K.models.load_model('/home/bmcfee/git/ismir2017_chords/data/model_struct_aug/fold00_weights.pkl',
                           custom_objects=dict(K=K), compile=False)

In [197]:
model.output_names

['chord_tag', 'chord_pitch', 'chord_root', 'chord_bass']

In [198]:
preds = model.predict(data['cqt/mag'])

In [200]:
[_.shape for _ in preds]

[(1, 2232, 170), (1, 2232, 12), (1, 2232, 13), (1, 2232, 13)]

In [204]:
ref_jam = jams.load('/home/bmcfee/data/eric_chords/references_v2/TRXBKDB149E3B7589E.jams')

In [203]:
chordino_jam = jams.load('/home/bmcfee/git/ismir2017_chords/data/chordino/TRXBKDB149E3B7589E.jams')

In [217]:
madmom_jam = jams.load('/home/bmcfee/git/ismir2017_chords/data/madmom/TRXBKDB149E3B7589E.jams')

In [205]:
data = pump.transform('/home/bmcfee/data/eric_chords/audio/TRXBKDB149E3B7589E.mp3')

In [206]:
tag_pred1, pc_pred1, root_pred1, bass_pred1 = cr1s.predict(data['cqt/mag'])

In [207]:
tag_pred2, pc_pred2, root_pred2, bass_pred2 = cr2s.predict(data['cqt/mag'])

In [208]:
ann1_noinv = op.inverse(tag_pred1[0])

In [209]:
ann1_inv = preds_to_inversions(op, tag_pred1[0], bass_pred1[0])

In [210]:
ann2_noinv = op.inverse(tag_pred2[0])

In [211]:
ann2_inv = preds_to_inversions(op, tag_pred2[0], bass_pred2[0])

In [212]:
pd.DataFrame.from_dict(dict(jams.eval.chord(ref_jam.annotations[0], ann1_noinv)), orient='index').T.sort_index(axis=1)

,majmin,majmin_inv,mirex,root,sevenths,sevenths_inv,tetrads,tetrads_inv,thirds,thirds_inv,triads,triads_inv
0,0.663337,0.58469,0.773219,0.663337,0.578541,0.493178,0.533024,0.454377,0.663337,0.58469,0.663337,0.58469


In [213]:
pd.DataFrame.from_dict(dict(jams.eval.chord(ref_jam.annotations[0], ann1_inv)), orient='index').T.sort_index(axis=1)

,majmin,majmin_inv,mirex,root,sevenths,sevenths_inv,tetrads,tetrads_inv,thirds,thirds_inv,triads,triads_inv
0,0.663337,0.576932,0.773219,0.663337,0.578541,0.487815,0.533024,0.449436,0.663337,0.576932,0.663337,0.576932


In [214]:
pd.DataFrame.from_dict(dict(jams.eval.chord(ref_jam.annotations[0], ann2_noinv)), orient='index').T.sort_index(axis=1)

,majmin,majmin_inv,mirex,root,sevenths,sevenths_inv,tetrads,tetrads_inv,thirds,thirds_inv,triads,triads_inv
0,0.773326,0.581846,0.828056,0.802215,0.716502,0.510696,0.661328,0.471713,0.773725,0.581846,0.773326,0.581846


In [215]:
pd.DataFrame.from_dict(dict(jams.eval.chord(ref_jam.annotations[0], ann2_inv)), orient='index').T.sort_index(axis=1)

,majmin,majmin_inv,mirex,root,sevenths,sevenths_inv,tetrads,tetrads_inv,thirds,thirds_inv,triads,triads_inv
0,0.773326,0.693663,0.828056,0.802215,0.716502,0.631769,0.661328,0.58326,0.773725,0.693663,0.773326,0.693663


In [218]:
pd.DataFrame.from_dict(dict(jams.eval.chord(ref_jam.annotations[0], madmom_jam.annotations[0])), orient='index').T.sort_index(axis=1)

,majmin,majmin_inv,mirex,root,sevenths,sevenths_inv,tetrads,tetrads_inv,thirds,thirds_inv,triads,triads_inv
0,0.78169,0.641576,0.809598,0.78169,0.370845,0.252472,0.341668,0.232609,0.78169,0.641576,0.78169,0.641576


In [216]:
pd.DataFrame.from_dict(dict(jams.eval.chord(ref_jam.annotations[0], chordino_jam.annotations[0])), orient='index').T.sort_index(axis=1)

,majmin,majmin_inv,mirex,root,sevenths,sevenths_inv,tetrads,tetrads_inv,thirds,thirds_inv,triads,triads_inv
0,0.427286,0.361316,0.608117,0.427286,0.363114,0.291511,0.389464,0.323494,0.427286,0.361316,0.427286,0.361316
